In [ ]:
!unzip /content/drive/MyDrive/Tr_Aug_rand.zip

Streaming output truncated to the last 5000 lines.
 extracting: Tr_Aug_rand/Writer9105/0010.png  
 extracting: Tr_Aug_rand/Writer9105/0011.png  
 extracting: Tr_Aug_rand/Writer9105/0012.png  
 extracting: Tr_Aug_rand/Writer9105/0013.png  
 extracting: Tr_Aug_rand/Writer9105/0014.png  
 extracting: Tr_Aug_rand/Writer9105/0015.png  
 extracting: Tr_Aug_rand/Writer9105/0016.png  
 extracting: Tr_Aug_rand/Writer9105/0017.png  
 extracting: Tr_Aug_rand/Writer9105/0018.png  
 extracting: Tr_Aug_rand/Writer9105/0019.png  
 extracting: Tr_Aug_rand/Writer9105/0020.png  
 extracting: Tr_Aug_rand/Writer9105/0021.png  
 extracting: Tr_Aug_rand/Writer9105/0022.png  
 extracting: Tr_Aug_rand/Writer9105/0023.png  
 extracting: Tr_Aug_rand/Writer9105/0024.png  
 extracting: Tr_Aug_rand/Writer9105/0025.png  
 extracting: Tr_Aug_rand/Writer9105/0026.png  
 extracting: Tr_Aug_rand/Writer9105/0027.png  
 extracting: Tr_Aug_rand/Writer9105/0028.png  
 extracting: Tr_Aug_rand/Writer9105/0029.png  
 extracti

In [ ]:
!zip -r /content/images.zip /content/images


  adding: content/images/ (stored 0%)
  adding: content/images/Writer9898/ (stored 0%)
  adding: content/images/Writer9898/0494.png (stored 0%)
  adding: content/images/Writer9898/0190.png (stored 0%)
  adding: content/images/Writer9898/0078.png (deflated 0%)
  adding: content/images/Writer9898/0234.png (stored 0%)
  adding: content/images/Writer9898/0439.png (stored 0%)
  adding: content/images/Writer9898/0239.png (deflated 1%)
  adding: content/images/Writer9898/0436.png (deflated 0%)
  adding: content/images/Writer9898/0192.png (stored 0%)
  adding: content/images/Writer9898/0362.png (stored 0%)
  adding: content/images/Writer9898/0238.png (stored 0%)
  adding: content/images/Writer9898/0463.png (stored 0%)
  adding: content/images/Writer9898/0245.png (stored 0%)
  adding: content/images/Writer9898/0420.png (stored 0%)
  adding: content/images/Writer9898/0387.png (stored 0%)
  adding: content/images/Writer9898/0105.png (stored 0%)
  adding: content/images/Writer9898/0074.png (stored

TRAIN FESHORDEGI

In [ ]:
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, Dense
import os
import cv2
from keras.callbacks import ModelCheckpoint
import scipy.io
from keras.layers import *
from keras.applications.densenet import DenseNet121
from keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D

#Load HTD feature descriptor
mat1 = scipy.io.loadmat('/content/Data_CERUG_Tr.mat')                                                  # Load HTD Train
HTDTr = mat1["Data1"]



lr_reducer    = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger    = CSVLogger('/content/drive/MyDrive/tr.csv')                             # Set name of log file

batch_size    = 64                                                                           #Set batch size
nb_epoch      = 50                                                                           #Set number of epochs
data_augmentation = False
VALIDATION_SPLIT  = 0.2         #######
img_rows, img_cols = 60, 180                                                                 #Set image row and column
img_channels  = 1                                                                            #Set image channel
INPUT_SHAPE   = (60, 180, 1)
HTD_SHAPE     = HTDTr.shape[1]
feat_SHAPE    = HTD_SHAPE
feat_shape=feat_SHAPE
TrNo          = HTDTr.shape[0]َ


###############################################################################
path_Tr     = '/content/Tr_Aug_rand'                           				# Set PathTrain
all_images  = []
y_train     = []
subjects    = os.listdir(path_Tr)
num_classes = len(subjects)     
print('Number of Classes: ', num_classes)
train_no    = 500                                                                              # Set number of train samples
test_no     = 30                                                                               # Set number of test samples
for number1 in range(0, num_classes):  
    print(number1)
    pathFold = (path_Tr + '//'+subjects[number1]  )
    files    = os.listdir(pathFold)
    numberOfsamples = len(files)
    for number2 in range(0, train_no):                                                         # train number
        pathImg = pathFold + '//'+ files[number2]


        img     = cv2.imread(pathImg , 0)
        img     = np.resize(img,(60, 180, 1))
        all_images.append(img)
        y_train.append(int(number1))

X_train = np.array(all_images)
y_train = np.array(y_train, dtype=np.uint16) 
print(X_train.shape)

import  tensorflow as tf

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, num_classes)
X_train = X_train.astype('float32')


# subtract mean and normalize

X_train   /= 255.0


base_model=tf.keras.applications.DenseNet121(weights=None
                                             ,include_top=False,input_shape=(60, 180, 1))

for layer in base_model.layers[:400]:
    layer.trainable = False 
x=base_model.output

auxiliary_input = Input(shape=(feat_shape,),name='aux_input')

x = Flatten()(x)

conFeats = tf.keras.layers.concatenate([x, auxiliary_input])
prediction=Dense(units=num_classes
,kernel_initializer="he_normal",
                      activation="softmax")(conFeats)
model =tf.keras.models.Model(inputs=[base_model.input,auxiliary_input ],outputs=prediction)


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


history = model.fit([X_train,HTDTr], Y_train,batch_size=8,
                    epochs=16, verbose="auto",
                    shuffle=True,callbacks=[lr_reducer, early_stopper, csv_logger])
model.save("/content/drive/MyDrive/model-deepze")

Exception ignored in: <function _xla_gc_callback at 0x7effb4463f40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


FileNotFoundError: ignored

TRAIN MOLTI

In [ ]:
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, Dense
import os
import cv2
from keras.callbacks import ModelCheckpoint
import scipy.io
from keras.layers import *
from keras.applications.densenet import DenseNet121
from keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

#Load HTD feature descriptor
mat1 = scipy.io.loadmat('/content/Data_CERUG_Tr.mat')                                                  # Load HTD Train
HTDTr = mat1["Data1"]
mat1=scipy.io.loadmat('/content/My_CERUG-EN_IMG_Tr_rand1.mat')
HTDTr1=mat1["W"]


csv_logger    = CSVLogger('tr.csv')                             # Set name of log file

batch_size    = 64                                                                           #Set batch size
nb_epoch      = 50                                                                           #Set number of epochs
data_augmentation = False
VALIDATION_SPLIT  = 0.2         #######
img_rows, img_cols = 60, 180                                                                 #Set image row and column
img_channels  = 1                                                                            #Set image channel
INPUT_SHAPE   = (60, 180, 1)
HTD_SHAPE     = HTDTr.shape[1]
feat_SHAPE    = HTD_SHAPE
feat_shape=feat_SHAPE
TrNo          = HTDTr.shape[0]
FEAT=HTDTr1.shape[1]

###############################################################################
path_Tr     = '/content/Tr_Aug_rand'                           				# Set PathTrain
all_images  = []
y_train     = []
subjects    = os.listdir(path_Tr)
num_classes = len(subjects)
print('Number of Classes: ', num_classes)
train_no    = 500                                                                              # Set number of train samples
test_no     = 30                                                                               # Set number of test samples
for number1 in range(0, num_classes):
    print(number1)
    pathFold = (path_Tr + '//'+subjects[number1]  )
    files    = os.listdir(pathFold)
    numberOfsamples = len(files)
    for number2 in range(0, train_no):                                                         # train number
        pathImg = pathFold + '//'+ files[number2]


        img     = cv2.imread(pathImg , 0)
        img     = np.resize(img,(60, 180, 1))
        all_images.append(img)
        y_train.append(int(number1))

X_train = np.array(all_images)
y_train = np.array(y_train, dtype=np.uint16)
print(X_train.shape)

import  tensorflow as tf

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, num_classes)
X_train = X_train.astype('float32')


# subtract mean and normalize

X_train   /= 255.0


base_model=tf.keras.applications.DenseNet121(weights=None
                                             ,include_top=False,input_shape=(60, 180, 1))
for layer in base_model.layers[:400]:
    layer.trainable = False
x=base_model.output

auxiliary_input = Input(shape=(feat_shape,),name='aux_input')
auxiliary_input1=Input(shape=(FEAT,),name='aux_input1')
x = Flatten()(x)
conFeats = tf.keras.layers.concatenate([x, auxiliary_input,auxiliary_input1])
prediction=Dense(units=num_classes
,kernel_initializer="he_normal",
                      activation="softmax")(conFeats)
model =tf.keras.models.Model(inputs=[base_model.input,auxiliary_input,auxiliary_input1 ],outputs=prediction)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.summary()


history = model.fit([X_train,HTDTr,HTDTr1], Y_train,batch_size=16,
                    epochs=20, verbose=1,
                    shuffle=True,callbacks=[csv_logger])

Number of Classes:  105
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
(52500, 60, 180, 1)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 180, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 66, 186, 1)  0           ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 30, 90, 64)   3136        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 30, 90, 64)   256         ['conv1/conv[0][0]']         

DEEEEEEEEP

In [ ]:
!unrar x "/content/drive/MyDrive/Line5Ptch.rar" "/content/tr"

Streaming output truncated to the last 5000 lines.
Extracting  /content/tr/Line5Ptch/Tr/546/0754.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0755.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0756.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0757.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0758.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0759.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0760.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0761.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0762.png                         87%  OK 
Extracting  /content/tr/Line5Ptch/Tr/546/0763.png                         87%  OK 
Extracting  /content/tr/Line5

In [ ]:
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, Dense
import os
import cv2
from keras.callbacks import ModelCheckpoint
import scipy.io
from keras.layers import *
from keras.applications.densenet import DenseNet121
from keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D

#Load HTD feature descriptor
#mat1 = scipy.io.loadmat('D:\\CVDataset\\Handwriting\\SOrce\\feshrdegi\\My_CERUG-EN_IMG_Tr_rand.mat')                                                  # Load HTD Train
#HTDTr = mat1["Data1"]




csv_logger    = CSVLogger('tr1.csv')                             # Set name of log file

                                                                           #Set number of epochs
data_augmentation = False
VALIDATION_SPLIT  = 0.2         #######
img_rows, img_cols = 60, 180                                                                 #Set image row and column
img_channels  = 1                                                                            #Set image channel
INPUT_SHAPE   = (60, 180, 1)
#HTD_SHAPE     = HTDTr.shape[1]
#feat_SHAPE    = HTD_SHAPE
#feat_shape=feat_SHAPE
#TrNo          = HTDTr.shape[0]


###############################################################################
path_Tr     = '/content/tr/Line5Ptch/Tr'                           				# Set PathTrain
all_images  = []
y_train     = []
subjects    = os.listdir(path_Tr)
num_classes = len(subjects)
print('Number of Classes: ', num_classes)
train_no    = 500                                                                              # Set number of train samples
test_no     = 30                                                                               # Set number of test samples
for number1 in range(0, num_classes):
    print(number1)
    pathFold = (path_Tr + '//'+subjects[number1]  )
    files    = os.listdir(pathFold)
    numberOfsamples = len(files)
    for number2 in range(0, train_no):                                                         # train number
        pathImg = pathFold + '//'+ files[number2]


        img     = cv2.imread(pathImg , 0)
        img     = np.resize(img,(60, 180, 1))
        all_images.append(img)
        y_train.append(int(number1))

X_train = np.array(all_images)
y_train = np.array(y_train, dtype=np.uint16)
print(X_train.shape)

import  tensorflow as tf

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, num_classes)
X_train = X_train.astype('float32')


# subtract mean and normalize

X_train   /= 255.0
#tf.keras.applications.DenseNet121

model=tf.tf.keras.applications.DenseNet121(
    include_top=True,
    weights=None,
    input_shape=(60,180,1),
    pooling=GlobalAveragePooling2D,
    classes=657,
    classifier_activation="softmax")

for layer in base_model.layers[:300]:  # و دستاست بزرگ برای iam 300
    layer.trainable = False
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


history = model.fit(X_train, Y_train,batch_size=32,
                    epochs=25, verbose=1,
                    shuffle=True,callbacks=[csv_logger])

Number of Classes:  105
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
(52500, 60, 180, 1)
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 180, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 66, 186, 1)   0           ['input_1[0][0]']                
                                  

3282/3282 [==============================] - 262s 64ms/step - loss: 2.9391 - accuracy: 0.2371 - lr: 0.0010
Epoch 2/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.9314 - accuracy: 0.7092

3282/3282 [==============================] - 205s 62ms/step - loss: 0.9314 - accuracy: 0.7092 - lr: 0.0010
Epoch 3/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.3653 - accuracy: 0.8866

3282/3282 [==============================] - 204s 62ms/step - loss: 0.3653 - accuracy: 0.8866 - lr: 0.0010
Epoch 4/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.1730 - accuracy: 0.9458

3282/3282 [==============================] - 203s 62ms/step - loss: 0.1730 - accuracy: 0.9458 - lr: 0.0010
Epoch 5/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.1065 - accuracy: 0.9671

...

3282/3282 [==============================] - 204s 62ms/step - loss: 0.0120 - accuracy: 0.9966 - lr: 0.0010
Epoch 22/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.0131 - accuracy: 0.9963

3282/3282 [==============================] - 205s 62ms/step - loss: 0.0131 - accuracy: 0.9963 - lr: 0.0010
Epoch 23/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.0108 - accuracy: 0.9970

3282/3282 [==============================] - 205s 62ms/step - loss: 0.0108 - accuracy: 0.9970 - lr: 0.0010
Epoch 24/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.0103 - accuracy: 0.9972

3282/3282 [==============================] - 205s 63ms/step - loss: 0.0103 - accuracy: 0.9972 - lr: 0.0010
Epoch 25/25
3281/3282 [============================>.] - ETA: 0s - loss: 0.0096 - accuracy: 0.9970

3282/3282 [==============================] - 205s 62ms/step - loss: 0.0096 - accuracy: 0.9970 - lr: 0.0010
